# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Create ETL pipeline for the business data from the API

### Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

### Request

In [2]:
import requests
import pandas as pd
import json

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

# keys.keys()

In [25]:
def yelp_request(term, location, yelp_key, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

In [5]:
request = yelp_request('pizza','baltimore', yelp_key)

In [13]:
request.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
# Show first item w/in list of businesses

request['businesses'][0]

{'id': '417YPonNwsTF-LKjGybNGA',
 'alias': 'angelis-pizzeria-baltimore-2',
 'name': "Angeli's Pizzeria",
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/285RaNTzD_rTbk2aYWC49w/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/angelis-pizzeria-baltimore-2?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
 'review_count': 252,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'},
  {'alias': 'sandwiches', 'title': 'Sandwiches'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.2853676681653, 'longitude': -76.6010601392803},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '413 S High St',
  'address2': '',
  'address3': None,
  'city': 'Baltimore',
  'zip_code': '21202',
  'country': 'US',
  'state': 'MD',
  'display_address': ['413 S High St', 'Baltimore, MD 21202']},
 'phone': '+14437087556',
 'display_phon

In [23]:
request_df = pd.DataFrame(request['businesses'])

In [26]:
request_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,417YPonNwsTF-LKjGybNGA,angelis-pizzeria-baltimore-2,Angeli's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/285RaN...,False,https://www.yelp.com/biz/angelis-pizzeria-balt...,252,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.2853676681653, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '413 S High St', 'address2': '', ...",+14437087556,(443) 708-7556,2746.815371
1,9PgU81DELsu8GaVX8FT45w,underground-pizza-baltimore-2,Underground Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/l2yfFl...,False,https://www.yelp.com/biz/underground-pizza-bal...,60,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 39.28907, 'longitude': -76.607268}",[delivery],NaN,"{'address1': '30 Market Pl', 'address2': '', '...",+14433476190,(443) 347-6190,2255.198296
2,BJrktyd8gy1bPd9flu54RA,isabellas-brick-oven-pizza-and-panini-baltimore,Isabella's Brick Oven Pizza and Panini,https://s3-media1.fl.yelpcdn.com/bphoto/5pf2ED...,False,https://www.yelp.com/biz/isabellas-brick-oven-...,531,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.2870444, 'longitude': -76.6020...","[delivery, pickup]",$$,"{'address1': '221 S High St', 'address2': '', ...",+14109628888,(410) 962-8888,2544.246639
3,1n30b0G2weApQ4oEfPMWIA,matthews-pizza-baltimore,Matthew's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/n9refU...,False,https://www.yelp.com/biz/matthews-pizza-baltim...,486,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 39.2861976623535, 'longitude': -7...",[delivery],$$,"{'address1': '3131 Eastern Ave', 'address2': '...",+14102768755,(410) 276-8755,4095.106366
4,I0GbQkQHF2sinkmG5vA3hg,bricknfire-at-baltimore-marriott-inner-harbor-...,BricknFire at Baltimore Marriott Inner Harbor ...,https://s3-media1.fl.yelpcdn.com/bphoto/wLbKlF...,False,https://www.yelp.com/biz/bricknfire-at-baltimo...,198,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.28685, 'longitude': -76.62138}","[delivery, pickup]",$$,"{'address1': '110 S Eutaw St', 'address2': Non...",+14437647847,(443) 764-7847,2690.520752


### Parse

In [ ]:
# your code to parse the result to make them easier to insert into the DB

def parse_results(results):

    return parsed_results

### Save

In [ ]:
def df_safe(csv filepath, parsed results):
    # your code to open the csv file, concat the current data, and save the data. 


- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [ ]:
from helper_funcs import *

In [ ]:


# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20

___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you ahve the process below 

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

- Write a function that takes a business id and makes a call to the API for reivews


- Write a function to parse out the relevant information from the reviews

- Write a function to save the parse data into a csv file containing all of the reviews. 

- Combine the functions above into a single script  

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 

- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20